In [2]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

## Data Preprocessing

In [3]:
def preprocess_data():
    """ Age preprocessing: Adding Age_is_missing col and filling the empty age with an estimate based on Sex and PClass """
    train_df['Age_Missing'] = train_df['Age'].isna().astype(int)
    test_df['Age_Missing'] = test_df['Age'].isna().astype(int)
    
    global_age = train_df['Age'].median()
    group_age = train_df.groupby(['Sex','Pclass'])['Age'].median()
    def estimate_age(row):
        if pd.isna(row['Age']):
            age = group_age.get((row['Sex'], row['Pclass']), np.nan)
            return age if not np.isnan(age) else global_age
        return row['Age']
    
    train_df['Age'] = train_df.apply(estimate_age, axis=1)
    test_df['Age'] = test_df.apply(estimate_age, axis=1)
    
    """ Adding a Family_size col that reflects sum of sibsp and parch """
    
    train_df['Family_Size'] = train_df['Parch'] + train_df['SibSp'] + 1
    test_df['Family_Size'] = test_df['Parch'] + test_df['SibSp'] + 1
    
    """ Fare preprocessing: Filling the empty fare with an estimate based on Sex and PClass """
    global_fare = train_df['Fare'].median()
    group_fare = train_df.groupby(['Sex', 'Pclass'])['Fare'].median()
    
    def estimate_fare(row):
        if pd.isna(row['Fare']):
            fare = group_fare.get((row['Sex'], row['Pclass']), np.nan)
            return fare if not pd.isna(fare) else global_fare
        return row['Fare']
    train_df['Fare'] = train_df.apply(estimate_fare, axis=1)
    test_df['Fare'] = test_df.apply(estimate_fare, axis=1)
    
    """ Change cabin to be the first letter of cabin and also fill with U if it is empty """
    train_df['Cabin'] = train_df['Cabin'].str[0].fillna('U')
    test_df['Cabin'] = test_df['Cabin'].str[0].fillna('U')
    
    """ If Embarked is empty, use 'U' """
    train_df['Embarked'] = train_df['Embarked'].fillna('U')
    test_df['Embarked'] = test_df['Embarked'].fillna('U')
    
    """ Convert Sex to numerical col (female=1, male=0)"""
    train_df['Sex'] = (train_df['Sex']=='female').astype(int)
    test_df['Sex'] = (test_df['Sex']=='female').astype(int)
    
    
    """ Convert other cat cols to numerical cols """
    cat_cols = ['Cabin', 'Embarked']
    x_train_cat = pd.get_dummies(train_df[cat_cols], prefix=cat_cols, dummy_na=False)
    x_test_cat = pd.get_dummies(test_df[cat_cols], prefix=cat_cols, dummy_na=False)
    x_test_cat = x_test_cat.reindex(columns=x_train_cat.columns, fill_value=0)
    
    """ Standardize some numerical cols """
    cols_to_standardize = ['Age', 'Fare', 'Family_Size']
    mu = train_df[cols_to_standardize].mean()
    std = train_df[cols_to_standardize].std(ddof=0)
    std = std.replace(0, 1.0)
    train_df[cols_to_standardize] = (train_df[cols_to_standardize]-mu)/std
    test_df[cols_to_standardize] = (test_df[cols_to_standardize]-mu)/std
    
    """ Select numerical cols that we want to use in logistic regression """
    num_cols = ['Sex', 'Age', 'Age_Missing', 'Family_Size', 'Fare']
    x_train_num = train_df[num_cols].astype(float)
    x_test_num = test_df[num_cols].astype(float)
    

    
    
    x_train = pd.concat([x_train_num, x_train_cat], axis=1)
    x_test = pd.concat([x_test_num, x_test_cat], axis=1)
    
    return x_train, x_test
    

## Performing data preprocessing and converting to NumPy

In [4]:
x_train_df, x_test_df = preprocess_data()

x_train = x_train_df.to_numpy(dtype=np.float64, copy=False)
x_test = x_test_df.to_numpy(dtype=np.float64, copy=False)
y_train = train_df['Survived'].to_numpy(dtype=np.float64, copy=False)


## Logistic Regression From Scratch

In [43]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def logistic_cost(X, y, w, b, lambda_):
    z = X @ w + b
    return (np.logaddexp(0, z) - y*z).mean()

def regularized_logistic_cost(X, y, w, b, lambda_):
    z = X @ w + b
    return (np.logaddexp(0, z) - y*z).mean() + (lambda_/(2*m))*np.dot(w, w)

def compute_gradient(X, y, w, b, lambda_):
    m, n = X.shape
    z = X @ w + b
    f_x = sigmoid(z)
    err = f_x - y
    d_j_b = err.mean()
    d_j_w = (X.T @ err) /m 
    return d_j_w, d_j_b

def compute_regularized_gradient(X, y, w, b, lambda_):
    m, n = X.shape
    z = X @ w + b
    f_x = sigmoid(z)
    err = f_x - y
    d_j_b = err.mean()
    d_j_w = ((X.T @ err) + lambda_*w) /m
    return d_j_w, d_j_b

def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_):
    w, b = w_in, b_in
    cost_history =[]
    for i in range(num_iters):
        d_j_w, d_j_b = gradient_function(X, y, w, b, lambda_)
        w  = w - alpha * d_j_w
        b = b - alpha * d_j_b
        cost = cost_function(X, y, w, b, lambda_)
        if i % 10000 == 0: 
            print(f"cost in iteration {i} = {cost}")
            cost_history.append(cost)
    
    return w, b, cost_history



def predict(X, w, b):
    z = X @ w + b
    f_x = sigmoid(z)
    return [int(x>=0.5) for x in f_x]


In [44]:
# Initialize parameters for non regularized logistic regression
m, n = x_train.shape
# w = np.zeros(n)
# b = 0
np.random.seed(1)
w = 0.01 * (np.random.rand(n) - 0.5)
b = -1

alpha = 0.001


""" Update w and b using gradient descent """
w, b, costs = gradient_descent(x_train, y_train, w, b, logistic_cost, compute_gradient, alpha, 100000, lambda_)


predicted_values = predict(x_train, w, b)
print('Train Accuracy: %f'%(np.mean(predicted_values == y_train) * 100))



cost in iteration 0 = 0.6980479313369613
cost in iteration 10000 = 0.5199743077136791
cost in iteration 20000 = 0.4828057758196072
cost in iteration 30000 = 0.4672616025323831
cost in iteration 40000 = 0.4595522879004116
cost in iteration 50000 = 0.45533215184578174
cost in iteration 60000 = 0.45284983170867016
cost in iteration 70000 = 0.4513006929463314
cost in iteration 80000 = 0.4502827331712428
cost in iteration 90000 = 0.44958232272471255
Train Accuracy: 80.134680


In [48]:
# Initialize parameters for regularized logistic regression
m, n = x_train.shape
# w = np.zeros(n)
# b = 0
np.random.seed(1)
w = 0.01 * (np.random.rand(n) - 0.5)
b = -1

alpha = 0.001
lambda_ = -1


""" Update w and b using gradient descent """
w, b, costs = gradient_descent(x_train, y_train, w, b, regularized_logistic_cost, compute_regularized_gradient, alpha, 100000, lambda_)

predicted_values = predict(x_train, w, b)
print('Train Accuracy: %f'%(np.mean(predicted_values == y_train) * 100))



cost in iteration 0 = 0.6980478604348902
cost in iteration 10000 = 0.5187560507134266
cost in iteration 20000 = 0.4803163592993963
cost in iteration 30000 = 0.46380899375407647
cost in iteration 40000 = 0.45538729597507
cost in iteration 50000 = 0.4506374179618552
cost in iteration 60000 = 0.4477558023924199
cost in iteration 70000 = 0.44590049750694194
cost in iteration 80000 = 0.44464329096253047
cost in iteration 90000 = 0.4437521809576839
Train Accuracy: 80.134680


## Predicting for the test set

In [55]:
predicted_test_values = predict(x_test, w, b)

print(predicted_test_values[:10])
test_df['PassengerId']

output_file = pd.DataFrame({
    'PassengerId': test_df['PassengerId'].astype(int).values,
    'Survived': predicted_test_values
})
output_file.to_csv('titanic_output.csv', index=False)

[0, 1, 0, 0, 1, 0, 1, 0, 1, 0]


## SKLearn baseline + tuning

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def fit_logreg(X, y, c=1.0, class_weight=None, max_iter=10000, random_state=20):
    clf = LogisticRegression(
        C=c, penalty="l2", solver = "lbfgs", class_weight=class_weight, 
        max_iter=max_iter, random_state=random_state)
    clf.fit(X, y)
    return clf


def train_accuracy(clf, X, y, threshold=0.5):
    p = clf.predict_proba(X)[:, 1]
    y_hat = (p>= threshold).astype(int)
    return accuracy_score(y, y_hat)


def predict_labels(clf, X, threshold=0.5):
    p = clf.predict_proba(X)[:, 1]
    return (p>=threshold).astype(int)    

def write_submission(lables, path="sklearn-output.csv"):
    output_file = pd.DataFrame({
        'PassengerId': test_df['PassengerId'].astype(int).values,
        'Survived': lables
    })
    output_file.to_csv(path, index=False)

In [26]:
clf = fit_logreg(x_train, y_train, c=0.45, max_iter=100000)
acc = train_accuracy(clf, x_train, y_train)
print(f'Train Accuracy: {acc}')
lables = predict_labels(clf, x_test)
write_submission(lables)

Train Accuracy: 0.8047138047138047
